# Reference-Based Evaluation of Audio Captioning Models

This notebook performs comprehensive evaluation using **reference-based metrics**:

- **BLEU (1-4)**: N-gram overlap with references
- **METEOR**: Alignment-based metric with synonyms consideration
- **ROUGE-L**: Longest common subsequence
- **CIDEr**: Consensus-based metric using TF-IDF

These metrics compare generated captions against human-written references, providing quantitative measures of caption quality.

## Step 1: Setup and Imports

In [ ]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

import sys
from pathlib import Path

# Add parent directory to path
project_root = Path('..').absolute()
sys.path.insert(0, str(project_root))

import torch
torch.cuda.empty_cache()
import gc
gc.collect()

import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from src.models import create_model
from src.dataset import create_dataloaders
from src.utils import load_vocab, set_seed, get_device
from src.model_configs import load_model_from_checkpoint
from src.reference_metrics import (
    compute_all_metrics,
    evaluate_captions,
    print_metrics,
    bleu_score,
    meteor_score,
    rouge_l_score,
    cider_score
)

print("✓ Imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Step 2: Test Reference Metrics on Examples

Let's first test the metrics on some example captions to understand what they measure.

In [ ]:
print("Testing reference-based metrics on example captions...\n")

# Example references (ground truth)
references = [
    "a dog is barking loudly in the backyard",
    "the dog barks outside in the yard",
    "loud barking from a dog in an outdoor area"
]

print("Reference captions:")
for i, ref in enumerate(references, 1):
    print(f"  {i}. {ref}")

print("\n" + "="*80)

# Test different candidate qualities
test_cases = [
    ("Perfect match", "a dog is barking loudly in the backyard"),
    ("Good paraphrase", "a dog barking in the yard"),
    ("Partial match", "a dog in the backyard"),
    ("Related but different", "a cat is meowing in the house"),
    ("Completely wrong", "cars driving on a highway")
]

for label, candidate in test_cases:
    print(f"\n{label}: '{candidate}'")
    print("-" * 80)
    
    metrics = compute_all_metrics(candidate, references)
    
    # Print in one line
    print(f"  BLEU-1: {metrics['BLEU-1']:.3f}  |  "
          f"BLEU-4: {metrics['BLEU-4']:.3f}  |  "
          f"METEOR: {metrics['METEOR']:.3f}  |  "
          f"ROUGE-L: {metrics['ROUGE-L']:.3f}  |  "
          f"CIDEr: {metrics['CIDEr']:.3f}")

print("\n" + "="*80)

## Step 3: Load Data and Model

In [ ]:
set_seed(42)
device = get_device()

print(f"Using device: {device}")

In [ ]:
# Load vocabulary
vocab = load_vocab('../vocab.json')
print(f"Vocabulary size: {len(vocab)}")

# Create reverse mapping
idx_to_word = {v: k for k, v in vocab.items()}

In [ ]:
# Load eval dataset
_, _, eval_dataset = create_dataloaders(
    train_captions='../data/train_captions.json',
    val_captions='../data/val_captions.json',
    eval_captions='../data/eval_captions.json',
    train_features_dir='../features/mel/',
    val_features_dir='../features/mel/',
    eval_features_dir='../features/mel_eval/',
    vocab=vocab,
    batch_size=8,
    num_workers=2
)

print(f"Eval dataset size: {len(eval_dataset)}")

## Step 4: Load Trained Models

We'll evaluate all available trained models.

In [ ]:
import os.path as osp

# Check for available checkpoints
checkpoint_dir = '../checkpoints'
available_checkpoints = []

if osp.exists(checkpoint_dir):
    for fname in os.listdir(checkpoint_dir):
        if fname.endswith('.pth'):
            available_checkpoints.append(osp.join(checkpoint_dir, fname))

print(f"Found {len(available_checkpoints)} checkpoint(s):")
for cp in available_checkpoints:
    print(f"  - {osp.basename(cp)}")

# Load all available models using automatic config detection
models = {}

for checkpoint_path in available_checkpoints:
    checkpoint_name = osp.basename(checkpoint_path)
    
    # Determine model name from checkpoint filename
    if 'baseline_small' in checkpoint_name:
        model_name = 'baseline_small'
    elif 'baseline' in checkpoint_name and 'improved' not in checkpoint_name:
        model_name = 'baseline'
    elif 'improved_baseline' in checkpoint_name:
        model_name = 'improved_baseline'
    elif 'attention_small' in checkpoint_name:
        model_name = 'attention_small'
    elif 'attention' in checkpoint_name:
        model_name = 'attention'
    elif 'transformer_small' in checkpoint_name:
        model_name = 'transformer_small'
    elif 'transformer' in checkpoint_name:
        model_name = 'transformer'
    else:
        # Try to use filename without extension
        model_name = checkpoint_name.replace('best_', '').replace('.pth', '')
    
    print(f"\nLoading {model_name} from {checkpoint_name}...")
    
    try:
        # Use automatic config detection
        model, config, checkpoint = load_model_from_checkpoint(
            checkpoint_path,
            vocab_size=len(vocab),
            device=device
        )
        
        model.eval()
        models[model_name] = model
        
        print(f"  ✓ Successfully loaded {model_name}")
        print(f"  Config: {config}")
        
    except Exception as e:
        print(f"  ✗ Failed to load {model_name}: {e}")
        continue

print(f"\n{'='*80}")
print(f"Successfully loaded {len(models)} model(s): {list(models.keys())}")
print(f"{'='*80}")

## Step 5: Generate Captions for All Models

In [ ]:
def decode_tokens(token_ids, vocab):
    """Convert token IDs to text"""
    idx_to_word = {v: k for k, v in vocab.items()}
    words = []
    for idx in token_ids:
        idx = idx.item() if torch.is_tensor(idx) else idx
        if idx == vocab['<eos>']:
            break
        if idx not in [vocab['<pad>'], vocab['<sos>'], 0]:
            words.append(idx_to_word.get(idx, '<unk>'))
    return ' '.join(words)


def generate_captions_for_model(model, eval_dataset, vocab, device, num_samples=100, 
                                 temperature=0.7, use_sampling=True):
    """
    Generate captions for evaluation dataset
    
    Args:
        model: Model to evaluate
        eval_dataset: Evaluation dataset
        vocab: Vocabulary
        device: Device
        num_samples: Number of samples to evaluate
        temperature: Sampling temperature (lower = more conservative)
        use_sampling: If False, use greedy decoding
    """
    model.eval()
    
    generated_captions = []
    reference_captions = []
    
    num_samples = min(num_samples, len(eval_dataset))
    
    for i in tqdm(range(num_samples), desc="Generating"):
        item = eval_dataset[i]
        mel = item['mel'].unsqueeze(0).to(device)
        
        with torch.no_grad():
            if use_sampling:
                ids = model.generate(
                    mel,
                    max_len=30,
                    sos_idx=vocab['<sos>'],
                    eos_idx=vocab['<eos>'],
                    temperature=temperature,
                    top_k=50,
                    top_p=0.9
                )
            else:
                # Greedy (very low temperature)
                ids = model.generate(
                    mel,
                    max_len=30,
                    sos_idx=vocab['<sos>'],
                    eos_idx=vocab['<eos>'],
                    temperature=0.1,
                    top_p=1.0
                )
        
        caption = decode_tokens(ids[0], vocab)
        generated_captions.append(caption)
        reference_captions.append(item['captions'])
    
    return generated_captions, reference_captions


# Generate captions for all models
print("Generating captions for all models...\n")

all_generated = {}
references = None  # Same for all models

NUM_EVAL_SAMPLES = 200  # Adjust based on your needs

for model_name, model in models.items():
    print(f"\nGenerating for {model_name}...")
    generated, refs = generate_captions_for_model(
        model, eval_dataset, vocab, device,
        num_samples=NUM_EVAL_SAMPLES,
        temperature=0.7,
        use_sampling=True
    )
    all_generated[model_name] = generated
    
    if references is None:
        references = refs
    
    print(f"  Generated {len(generated)} captions")

print("\n✓ Caption generation complete!")

## Step 6: Evaluate with Reference-Based Metrics

In [ ]:
print("Evaluating all models with reference-based metrics...\n")

all_results = {}

for model_name, generated in all_generated.items():
    print(f"\n{'='*60}")
    print(f"Evaluating {model_name.upper()}")
    print(f"{'='*60}")
    
    metrics = evaluate_captions(generated, references)
    all_results[model_name] = metrics
    
    print_metrics(metrics, f"{model_name.upper()} Results")

print("\n✓ Evaluation complete!")

## Step 7: Comparison Table and Visualization

In [ ]:
# Create comparison DataFrame
df = pd.DataFrame(all_results).T

print("\n" + "="*100)
print("MODEL COMPARISON - REFERENCE-BASED METRICS")
print("="*100)
print(df.to_string())
print("="*100)

# Save to CSV
df.to_csv('../results/reference_based_metrics_comparison.csv')
print("\n✓ Saved to results/reference_based_metrics_comparison.csv")

In [ ]:
# Visualize metrics
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Reference-Based Metrics Comparison', fontsize=16, fontweight='bold')

metrics_to_plot = ['BLEU-1', 'BLEU-2', 'BLEU-3', 'BLEU-4', 'METEOR', 'ROUGE-L']
colors = plt.cm.Set3(range(len(models)))

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 3, idx % 3]
    
    values = [all_results[model][metric] for model in models.keys()]
    bars = ax.bar(range(len(models)), values, color=colors)
    
    ax.set_xticks(range(len(models)))
    ax.set_xticklabels(models.keys(), rotation=45, ha='right')
    ax.set_ylabel('Score')
    ax.set_title(metric, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('../results/reference_metrics_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved visualization to results/reference_metrics_comparison.png")

In [ ]:
# CIDEr comparison (separate plot as it has different scale)
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

cider_values = [all_results[model]['CIDEr'] for model in models.keys()]
bars = ax.bar(range(len(models)), cider_values, color=colors)

ax.set_xticks(range(len(models)))
ax.set_xticklabels(models.keys(), rotation=45, ha='right')
ax.set_ylabel('CIDEr Score', fontsize=12)
ax.set_title('CIDEr Score Comparison', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.3f}',
            ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('../results/cider_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved CIDEr visualization")

## Step 8: Sample Predictions with Metrics

In [ ]:
print("\n" + "="*100)
print("SAMPLE PREDICTIONS WITH REFERENCE-BASED SCORES")
print("="*100)

num_samples_to_show = 5

for i in range(num_samples_to_show):
    print(f"\n{'='*100}")
    print(f"SAMPLE {i+1}: {eval_dataset[i]['fname']}")
    print(f"{'='*100}")
    
    # Show references
    print("\nReferences:")
    for j, ref in enumerate(references[i], 1):
        print(f"  {j}. {ref}")
    
    print("\n" + "-"*100)
    
    # Show each model's prediction and scores
    for model_name in models.keys():
        candidate = all_generated[model_name][i]
        metrics = compute_all_metrics(candidate, references[i])
        
        print(f"\n{model_name.upper()}:")
        print(f"  Generated: {candidate}")
        print(f"  Scores: BLEU-4={metrics['BLEU-4']:.3f} | "
              f"METEOR={metrics['METEOR']:.3f} | "
              f"ROUGE-L={metrics['ROUGE-L']:.3f} | "
              f"CIDEr={metrics['CIDEr']:.3f}")

print("\n" + "="*100)

## Step 9: Statistical Analysis

In [ ]:
# Compute per-sample metrics for statistical analysis
print("Computing per-sample metrics for statistical analysis...\n")

from src.reference_metrics import _compute_doc_freq

# Compute doc freq once
doc_freq = _compute_doc_freq(references)
num_docs = len(references)

per_sample_metrics = {}

for model_name, generated in all_generated.items():
    print(f"Processing {model_name}...")
    
    sample_metrics = []
    for candidate, refs in zip(generated, references):
        metrics = compute_all_metrics(candidate, refs, doc_freq, num_docs)
        sample_metrics.append(metrics)
    
    per_sample_metrics[model_name] = sample_metrics

print("\n✓ Per-sample metrics computed")

In [ ]:
# Statistical summary
print("\n" + "="*100)
print("STATISTICAL SUMMARY (Mean ± Std)")
print("="*100)

for metric in ['BLEU-4', 'METEOR', 'ROUGE-L', 'CIDEr']:
    print(f"\n{metric}:")
    print("-" * 60)
    
    for model_name in models.keys():
        scores = [m[metric] for m in per_sample_metrics[model_name]]
        mean = np.mean(scores)
        std = np.std(scores)
        print(f"  {model_name:.<30} {mean:.4f} ± {std:.4f}")

print("\n" + "="*100)

In [ ]:
# Box plots for distribution visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Score Distributions Across Models', fontsize=16, fontweight='bold')

metrics_for_box = ['BLEU-4', 'METEOR', 'ROUGE-L', 'CIDEr']

for idx, metric in enumerate(metrics_for_box):
    ax = axes[idx // 2, idx % 2]
    
    data = []
    labels = []
    
    for model_name in models.keys():
        scores = [m[metric] for m in per_sample_metrics[model_name]]
        data.append(scores)
        labels.append(model_name)
    
    bp = ax.boxplot(data, labels=labels, patch_artist=True)
    
    # Color boxes
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
    
    ax.set_ylabel('Score', fontsize=11)
    ax.set_title(metric, fontweight='bold', fontsize=12)
    ax.grid(axis='y', alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../results/score_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved distribution plots")

## Step 10: Save All Results

In [ ]:
# Save comprehensive results
results_to_save = {
    'num_samples': NUM_EVAL_SAMPLES,
    'models_evaluated': list(models.keys()),
    'averaged_metrics': all_results,
    'per_sample_metrics': per_sample_metrics,
    'generated_captions': all_generated,
    'reference_captions': references
}

# Save as JSON
with open('../results/reference_based_evaluation_complete.json', 'w') as f:
    json.dump(results_to_save, f, indent=2)

print("✓ Saved complete results to results/reference_based_evaluation_complete.json")

# Save summary
summary = {
    'num_samples': NUM_EVAL_SAMPLES,
    'models': list(models.keys()),
    'metrics': all_results
}

with open('../results/reference_metrics_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("✓ Saved summary to results/reference_metrics_summary.json")

## Summary

### Metrics Explained:

1. **BLEU (1-4)**: Measures n-gram precision (1-4 words)
   - Higher = more word overlap with references
   - BLEU-4 most commonly used
   - Range: 0-1 (higher is better)

2. **METEOR**: Alignment-based with stemming/synonyms
   - Balances precision and recall
   - More flexible than BLEU
   - Range: 0-1 (higher is better)

3. **ROUGE-L**: Longest common subsequence
   - Captures sentence-level structure
   - Order-aware unlike BLEU
   - Range: 0-1 (higher is better)

4. **CIDEr**: Consensus-based using TF-IDF
   - Weights rare/informative words higher
   - Standard for image/audio captioning
   - Range: 0-10 typically (higher is better)

### What Good Scores Look Like:

- **BLEU-4**: >0.15 (good), >0.25 (very good)
- **METEOR**: >0.20 (good), >0.30 (very good)
- **ROUGE-L**: >0.35 (good), >0.45 (very good)
- **CIDEr**: >0.5 (good), >1.0 (very good)

### Files Generated:

- `results/reference_based_metrics_comparison.csv` - Comparison table
- `results/reference_metrics_comparison.png` - Metrics visualization
- `results/cider_comparison.png` - CIDEr comparison
- `results/score_distributions.png` - Box plots
- `results/reference_based_evaluation_complete.json` - Full results
- `results/reference_metrics_summary.json` - Summary